# Data preprocessing

Man Ho Wong | March 21, 2022


In [1]:
# Python libraries and packages required
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import pprint                    # For pretty printing
import pylangacq                 # For reading CHAT files
from tqdm import tqdm            # For showing progress bar

# use ggplot style for figures
plt.style.use('ggplot')

# Unpickle data
data = pickle.load(open('../data/childes/corpus_info.pkl', 'rb'))
data_idx = data[1]

# Pretty printing for better readability
# With this, you can print dict in compact format instead of one item
#   per line. Items will be in alphabetical order. Nested Dict will be printed 
#   with suitable indentation. It will also print Counter in descending order.
#   To use this, call cp.pprint()
cp = pprint.PrettyPrinter(compact=True, sort_dicts=True)

# Function to get all labels of a given variable (e.g. 'mot_edu')
def get_labels(var):
    labels_by_corpus = {}
    corpus_set = set(data_idx.corpus)
    for c in corpus_set:
        labels_by_corpus[c] = set(data_idx[var][data_idx.corpus==c])
    return labels_by_corpus

`data_idx` is a `DataFrame` created by [data_curation.ipynb](Data_Science/Child-Vocab-Development/codes/data_curation.ipynb). It contains basic information about the files in the curated dataset. 

In [2]:
data_idx

,file_path,corpus,year,participants,name,age_d,age_m,sex,group,ses,mot_edu,situation,activities,study_type
0,../data/childes/Bates/Free20/amy.cha,Bates,NaN,"{CHI, MOT}",Target_Child,600,20.0,female,TD,MC,unspecified,unspecified,unspecified,"cross, toyplay, TD"
1,../data/childes/Bates/Free20/betty.cha,Bates,NaN,"{CHI, MOT}",Betty,600,20.0,female,TD,MC,unspecified,unspecified,unspecified,"cross, toyplay, TD"
2,../data/childes/Bates/Free20/chuck.cha,Bates,NaN,"{CHI, MOT}",Chuck,600,20.0,male,TD,MC,unspecified,unspecified,unspecified,"cross, toyplay, TD"
3,../data/childes/Bates/Free20/doug.cha,Bates,NaN,"{CHI, MOT}",Doug,600,20.0,male,TD,MC,unspecified,unspecified,unspecified,"cross, toyplay, TD"
4,../data/childes/Bates/Free20/ed.cha,Bates,NaN,"{CHI, MOT}",Ed,600,20.0,male,TD,MC,unspecified,unspecified,unspecified,"cross, toyplay, TD"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2606,../data/childes/VanHouten/Twos/teaching/parkt.cha,VanHouten,NaN,"{CHI, MOT}",Matthew,840,28.0,male,MOT_Older,unspecified,unspecified,Teaching. Mother and Child sit on floor. First...,unspecified,"cross, toyplay, TD"
2607,../data/childes/VanHouten/Twos/teaching/pricet...,VanHouten,NaN,"{CHI, MOT}",Peter,840,28.0,male,MOT_adolescent,unspecified,unspecified,Teaching. Mother and Child sit on the floor in...,unspecified,"cross, toyplay, TD"
2608,../data/childes/VanHouten/Twos/teaching/raidt.cha,VanHouten,NaN,"{CHI, MOT}",Tommy,840,28.0,male,MOT_older,unspecified,unspecified,"Teaching. Mother , Child and Father sit in liv...",unspecified,"cross, toyplay, TD"
2609,../data/childes/VanHouten/Twos/teaching/riott.cha,VanHouten,NaN,"{CHI, MOT}",Robert,840,28.0,male,MOT_Adolescent,unspecified,unspecified,Teaching. Mother and Child sit on the living r...,unspecified,"cross, toyplay, TD"


# Data cleaning and integration

As mentioned in `data_curation.ipynb`, not all corpora use the same labels for some variables. For example, some corpora use school grade year to define mother's education (`mot_edu`), and some use level of education (e.g. 'college'):

In [3]:
mot_edu_labels = set(data_idx.mot_edu)
cp.pprint(mot_edu_labels)  # Print compactly

{'**', '10', '10 , GED', '102', '11', '11 , GED', '11+', '12', '12+', '13',
 '13+', '14', '15', '16', '6', '7', '8', '9', "Associate's_Degree", 'College',
 'College_Doctoral', 'College_J.D.', "College_Master's", 'High_School_Diploma',
 'MOT_1', 'MOT_2', 'MOT_3', 'Some_College', 'XX', 'almost 12', 'unspecified'}


Besides `mot_edu`, other variables with different labels used in different corpora include:  
- `group` (child's developmental group, e.g. typically developed)
- `ses` (Child's SES (socioeconomic status; same as mother's))
- `situation` Situation of recording (e.g. play session)
- `activities` (Activities during recording, e.g. Toy play)
- `study_type` type of study (e.g. longitudinal study)
    
I will change the labels for some of these variables, so that all corpora use the same set of labels.

Below is a list of homepages for all the corpora appeared in the curated dataset. You can find the basic information about the corpus, such as variable and label definitions, on its homepage and in the linked publications. I will check these pages when the variable labels are ambiguous.

In [4]:
corpus_homepages = {}
corpus_set = set(data_idx.corpus)

for c in corpus_set:
    url = "https://childes.talkbank.org/access/Eng-NA/" + c + ".html"
    corpus_homepages[c] = url

cp.pprint(corpus_homepages)

{'Bates': 'https://childes.talkbank.org/access/Eng-NA/Bates.html',
 'Bernstein': 'https://childes.talkbank.org/access/Eng-NA/Bernstein.html',
 'Brown': 'https://childes.talkbank.org/access/Eng-NA/Brown.html',
 'Clark': 'https://childes.talkbank.org/access/Eng-NA/Clark.html',
 'Demetras2': 'https://childes.talkbank.org/access/Eng-NA/Demetras2.html',
 'Gleason': 'https://childes.talkbank.org/access/Eng-NA/Gleason.html',
 'HSLLD': 'https://childes.talkbank.org/access/Eng-NA/HSLLD.html',
 'Hall': 'https://childes.talkbank.org/access/Eng-NA/Hall.html',
 'Hicks': 'https://childes.talkbank.org/access/Eng-NA/Hicks.html',
 'Nelson': 'https://childes.talkbank.org/access/Eng-NA/Nelson.html',
 'NewmanRatner': 'https://childes.talkbank.org/access/Eng-NA/NewmanRatner.html',
 'Post': 'https://childes.talkbank.org/access/Eng-NA/Post.html',
 'VanHouten': 'https://childes.talkbank.org/access/Eng-NA/VanHouten.html'}


### `mot_edu`

I will start with the 'simplest' variable, `mot_edu`.
First, let's find out which of the above labels each corpus uses for `mot_edu`. I will use a self-defined function, `get_labels()` (see header section of this notebook):

In [5]:
cp.pprint(get_labels('mot_edu'))  # get_labels() is defined in header section

{'Bates': {'unspecified'},
 'Bernstein': {'unspecified'},
 'Brown': {'unspecified'},
 'Clark': {'unspecified'},
 'Demetras2': {'unspecified'},
 'Gleason': {'unspecified'},
 'HSLLD': {'**', '10', '10 , GED', '102', '11', '11 , GED', '11+', '12', '12+',
           '13', '13+', '14', '15', '16', '6', '7', '8', '9', 'XX', 'almost 12',
           'unspecified'},
 'Hall': {'unspecified'},
 'Hicks': {'unspecified'},
 'Nelson': {'unspecified'},
 'NewmanRatner': {"Associate's_Degree", 'College', 'College_Doctoral',
                  'College_J.D.', "College_Master's", 'High_School_Diploma',
                  'Some_College', 'unspecified'},
 'Post': {'unspecified'},
 'VanHouten': {'MOT_1', 'unspecified', 'MOT_2', 'MOT_3'}}


--
  
As shown here, different classifications of education level are used in three corpora (HSLLD, NewmanRatner and VanHouten). HSLLD has a very detailed classification while VanHouten has only three classes. To merge all the labels across different corpora, I will use a less detailed classification without losing information needed for this project.

The labels used in HSLLD correspond to the school grade year the mother achieved. In the U.S. educational system, 'GED' is equivalent to completing grade 12, so I will merge these two classes as one. The labels '13' and beyond correspond to some post-secondary education (e.g. vocational training).

According to VanHouten's documentation, mother’s education is classified as MOT_1 (completed junior high), MOT_2 (completed high school) or MOT_3 (some post-secondary education).

I will merge the current labels into the following labels according to their common definitions: 

- `JH-`, `HS-`, `HS`, `HS+` for:
    - before completing junior high (before grade 9),
    - before completing high school (before GED or grade 12),
    - completed high school (GED or grade 12), and
    - some post-secondary education (after high school).

- `AD`, `UG-`, `UG` for:
    - associate's degree,
    - some undergraduate education, and
    - bachelor's degree.

- `MS`, `JD`, `DR` for:
    - master's degree,
    - juris doctor degree, and
    - doctoral degree.
    
I will map the current `mot_edu` labels to the above new labels with a `dictionary` and update `data_idx` with the new labels:

In [6]:
# Function to update 'mot_edu'
def update_mot_edu(data):
    label = data
    #' mot_edu' label mapping
    EDU_DICT = {
              'JH-':['6','7','8'],
              'HS-':['9','10','11','11+','almost 12','MOT_1'],
              'HS':['10 , GED','11 , GED','12','12+',
                    'High_School_Diploma','MOT_2'],
              'HS+':['13','13+','14','15','16','MOT_3'],
              'AD':["Associate's_Degree"],
              'UG-':['Some_College'],
              'UG':['College'], 
              'MS':["College_Master's"],
              'JD':['College_J.D.'],    
              'DR':['College_Doctoral'],
              'unspecified':['**','102','XX','unspecified']
               }
    
    for key in EDU_DICT:
        if data in EDU_DICT[key]:
            label = key
    return label

# Update 'mot_edu'
data_idx.mot_edu = data_idx.mot_edu.map(update_mot_edu)

# Check if update was successful
cp.pprint(get_labels('mot_edu'))

{'Bates': {'unspecified'},
 'Bernstein': {'unspecified'},
 'Brown': {'unspecified'},
 'Clark': {'unspecified'},
 'Demetras2': {'unspecified'},
 'Gleason': {'unspecified'},
 'HSLLD': {'HS', 'unspecified', 'HS-', 'JH-', 'HS+'},
 'Hall': {'unspecified'},
 'Hicks': {'unspecified'},
 'Nelson': {'unspecified'},
 'NewmanRatner': {'UG', 'HS', 'unspecified', 'DR', 'UG-', 'AD', 'MS', 'JD'},
 'Post': {'unspecified'},
 'VanHouten': {'HS', 'unspecified', 'HS+', 'HS-'}}


--

All labels for `mot_edu` were updated sucessfully!

### `group`

Below are the labels used in different corpora:

In [7]:
cp.pprint(get_labels('group'))

{'Bates': {'TD'},
 'Bernstein': {'TD', 'unspecified'},
 'Brown': {'TD', 'unspecified'},
 'Clark': {'TD'},
 'Demetras2': {'TD', 'unspecified'},
 'Gleason': {'TD', 'unspecified', 'typical', 'normal'},
 'HSLLD': {'unspecified'},
 'Hall': {'TD', 'unspecified', 'White,UC'},
 'Hicks': {'unspecified'},
 'Nelson': {'unspecified'},
 'NewmanRatner': {'TD'},
 'Post': {'TD'},
 'VanHouten': {'MOT_Adolescent', 'MOT_Adolescent_', 'MOT_Older', 'MOT_Older_',
               'MOT_adolescent', 'MOT_older', 'TD', 'unspecified'}}


--

As shown above, the `group` information of each file is not properly speficied in every corpus: 
- According the documentations of the corpora, all children in the dataset should be typically developed (`TD` in `group`). However, not all corpora put this information in the `group` field in the header of each file. Therefore, I will change the `unspecified` label to `TD`.
- in the Gleason corpus, labels `normal` and `typical` are used in additon to `TD`. I will change all labels to `TD`.
- In the Hall corpus, the labels for `group` and `ses` were switched in some files (e.g. `White,UC` should be a label for `ses`). I will switch them first and change all `group` labels to `TD`.
- In the VanHouten corpus, all children are `TD`, but it also contains data from adolescent mothers which I will not use for this project. I will first remove these files and change all labels to `TD`.

Let's take a quick look at the Hall corpus first:

In [8]:
labels = ['TD', 'typical', 'normal']
data_idx[['group','ses','study_type']][(data_idx.corpus=='Hall') & 
                                       ~(data_idx.group.isin(labels))]

,group,ses,study_type
1471,unspecified,"Black,WC","cross, everyday, AAE"
1472,unspecified,"Black,WC","cross, everyday, AAE"
1473,unspecified,"Black,WC","cross, everyday, AAE"
1474,unspecified,"Black,WC","cross, everyday, AAE"
1475,unspecified,"Black,WC","cross, everyday, AAE"
1476,unspecified,"Black,WC","cross, everyday, AAE"
1477,unspecified,"Black,WC","cross, everyday, AAE"
1478,unspecified,"Black,WC","cross, everyday, AAE"
1479,unspecified,"Black,WC","cross, everyday, AAE"
1480,unspecified,"Black,WC","cross, everyday, AAE"


As shown above in row 1482 and 1485, the label `White,UC` (i.e. upper class white) is actually a label for `ses`. I will copy the labels to `ses` of the corresponding rows:

In [9]:
data_idx.loc[data_idx.group=='White,UC','ses'] = 'White,UC'

# Check if ses labels were updated
data_idx.loc[[1482,1485],['group','ses','study_type']]


,group,ses,study_type
1482,"White,UC","White,UC","cross, everyday, TD"
1485,"White,UC","White,UC","cross, everyday, TD"


Note that the `group` labels are still `White,UC`, but I will change them to `TD` together with other corpora later.  
Next, I will drop the data collected from adolescent mothers in the VanHouten corpus:

In [10]:
drop_labels = ['MOT_Adolescent', 'MOT_Adolescent_', 'MOT_adolescent']
data_idx.drop(data_idx[data_idx.group.isin(drop_labels)].index, inplace=True)

# Check current labels
cp.pprint(get_labels('group'))

{'Bates': {'TD'},
 'Bernstein': {'TD', 'unspecified'},
 'Brown': {'TD', 'unspecified'},
 'Clark': {'TD'},
 'Demetras2': {'TD', 'unspecified'},
 'Gleason': {'TD', 'unspecified', 'typical', 'normal'},
 'HSLLD': {'unspecified'},
 'Hall': {'TD', 'unspecified', 'White,UC'},
 'Hicks': {'unspecified'},
 'Nelson': {'unspecified'},
 'NewmanRatner': {'TD'},
 'Post': {'TD'},
 'VanHouten': {'MOT_Older', 'unspecified', 'TD', 'MOT_older', 'MOT_Older_'}}


--

Finally, we are ready to change all the labels to `TD`!

In [11]:
# Change all 'group' label to 
data_idx['group'] = 'TD'

# Check updated labels
cp.pprint(get_labels('group'))

{'Bates': {'TD'},
 'Bernstein': {'TD'},
 'Brown': {'TD'},
 'Clark': {'TD'},
 'Demetras2': {'TD'},
 'Gleason': {'TD'},
 'HSLLD': {'TD'},
 'Hall': {'TD'},
 'Hicks': {'TD'},
 'Nelson': {'TD'},
 'NewmanRatner': {'TD'},
 'Post': {'TD'},
 'VanHouten': {'TD'}}


--

### `ses`

Below are the labels for `ses`:

In [12]:
cp.pprint(get_labels('ses'))

{'Bates': {'MC'},
 'Bernstein': {'unspecified', 'MC'},
 'Brown': {'unspecified', 'MC'},
 'Clark': {'UC'},
 'Demetras2': {'unspecified', 'WC'},
 'Gleason': {'unspecified', 'MC'},
 'HSLLD': {'unspecified'},
 'Hall': {'unspecified', 'Black,UC', 'White,WC', 'Black,WC', 'White,UC'},
 'Hicks': {'LI', 'unspecified'},
 'Nelson': {'unspecified', 'MC'},
 'NewmanRatner': {'unspecified'},
 'Post': {'WC'},
 'VanHouten': {'unspecified'}}


--

In these corpora, `WC`, `MC`, `UC` and `LI` mean 'working class', 'middle class', 'upper class' and 'low income' respectively. Different labels are used in different corpora:
- In the Hall corpus, each class of `WC` and `UC` is subdivided into two racial groups, `Black` and `White`. Since the primary goal of this project is to investigate the effects of SES and mother's education on vocabulary development, effects from other factors such as race will not be studied. I will merge the labels for different racial groups but same SES class together to get a larger sample size for each class. Although race may play an important role in children's language development, I hope that the balanced distribution of samples from different racial groups in the Hall Corpus will minimize the effects of race on my analysis.
- In the Hicks corpus, `LI` ('low income') is used instead of common SES class such as `WC`. Since the majority of low-come families presumably belong to the working class, I will change the label `LI` to `WC`. - Besides, the children in the subdirectories '1st', '2nd', and '5th' were considered coming from `MC` families. The children in the subdirectory 'del' were from lower class (presumably `WC`) families. I will assigned the missed labels accordingly.
- It is stated on the homepage of the Bernstein corpus that "the mothers were all college-educated women, who were native-born Americans with white-collar husband". Since all families in the study have similar background and some of the files were labeled `MC`, it is very likely that other files should also be labeled `MC`.
- The Brown corpus contains data from three children: Adam (`MC`), Sarah (`WC`) and Eve (`unspecified`). I will assigned the missed labels accordingly.
- All children in the Demetras2 corpus were from `WC` families.
- All children in the Gleason corpus were from `MC` families.
- All children in the HSLLD corpus were from low-income families. I will use the label `WC`.
- There is only one child in the Nelson corpus and she came from a `MC` family.

There are two corpora where no SES class information could be found:
- NewmanRatner corpus
- VanHouten corpus: Only Hollingshead index (a measurement of SES) is provided.

Let's begin cleaning up the `ses` labels:

In [14]:
# Function to merge 'ses' labels
def merge_ses(data):
    label = data
    # 'ses' label mapping
    SES_DICT = {'WC':['WC', 'Black,WC', 'White,WC', 'LI'],
                'UC':['UC', 'Black,UC', 'White,UC']}
    for key in SES_DICT:
        if data in SES_DICT[key]:
            label = key
    return label

# merge 'ses' labels in Hall and Hicks corpus: 
data_idx.ses = data_idx.ses.map(merge_ses)

# Update labels in Hicks corpus
for folder in ['1st','2nd','5th']:
    data_idx.loc[((data_idx.corpus=='Hicks') & 
                  (data_idx.file_path.map(lambda x: folder in x))),'ses'] = 'MC'
data_idx.loc[((data_idx.corpus=='Hicks') & 
              (data_idx.file_path.map(lambda x: 'del' in x))),'ses'] = 'WC'

# Update labels in Brown corpus
data_idx.loc[((data_idx.corpus=='Brown')& (data_idx.name=='Adam')),'ses'] = 'MC'
data_idx.loc[((data_idx.corpus=='Brown')&(data_idx.name=='Sarah')),'ses'] = 'WC'

# Update labels in other corpora
data_idx.loc[data_idx.corpus=='Bernstein','ses'] = 'MC'
data_idx.loc[data_idx.corpus=='Demetras2','ses'] = 'WC'
data_idx.loc[data_idx.corpus=='Gleason','ses'] = 'MC'
data_idx.loc[data_idx.corpus=='HSLLD','ses'] = 'WC'
data_idx.loc[data_idx.corpus=='Nelson','ses'] = 'MC'

# # Check updated labels
cp.pprint(get_labels('ses'))

{'Bates': {'MC'},
 'Bernstein': {'MC'},
 'Brown': {'unspecified', 'MC', 'WC'},
 'Clark': {'UC'},
 'Demetras2': {'WC'},
 'Gleason': {'MC'},
 'HSLLD': {'WC'},
 'Hall': {'unspecified', 'UC', 'WC'},
 'Hicks': {'unspecified', 'MC', 'WC'},
 'Nelson': {'MC'},
 'NewmanRatner': {'unspecified'},
 'Post': {'WC'},
 'VanHouten': {'unspecified'}}


### `situation`

In [15]:
# cp.pprint(get_labels('activities'))

# for corpus in corpus_set:
#     for labels in get_labels('activities')[corpus]:
#         if 'reading' in labels:
#             print(corpus)
            
keywords = []            
for f in data_idx.situation:
    keywords.extend(f.lower().split())

Counter(keywords).most_common()

[('unspecified', 1299),
 ('home', 658),
 ('visit', 561),
 ('the', 439),
 ('and', 423),
 ('on', 397),
 ('in', 339),
 (',', 315),
 ('chi', 277),
 ('mot', 242),
 ('with', 240),
 ('is', 196),
 ('place', 175),
 ('to', 164),
 ('living', 158),
 ('1', 144),
 ('3', 143),
 ('mother', 142),
 ('room', 140),
 ('kitchen', 139),
 ('took', 137),
 ('sitting', 136),
 ('of', 135),
 ('at', 126),
 ('floor', 123),
 ('sarah', 122),
 ('a', 121),
 ('first', 120),
 ('session', 111),
 ('visit;', 110),
 ('second', 101),
 ('child', 101),
 ('table', 93),
 ('are', 90),
 ('play', 82),
 ('couch', 79),
 ('toys', 76),
 ('playing', 74),
 ('her', 72),
 ('2', 71),
 ('sat', 66),
 ('next', 65),
 ('1;', 60),
 ('room;', 56),
 ('book', 49),
 ('playroom', 48),
 ('3;', 48),
 ('as', 47),
 ('toy', 47),
 ('(lab)', 46),
 ('chair', 45),
 ('freeplay', 45),
 ('before', 44),
 ('sits', 44),
 ('sit', 44),
 ('tape', 43),
 ('time', 43),
 ('meal', 43),
 ('takes', 41),
 ("mot's", 40),
 ('lap', 39),
 ('afternoon', 38),
 ('reading', 36),
 ('bed'

In [ ]:
f = data_idx.file_path[data_idx.corpus=='Bernstein']
reader = pylangacq.Reader.from_files(f)

In [ ]:
group_df = pd.DataFrame()
group_df['name'] = [reader.headers()[file]['Participants']['CHI']['name'] for file in range(len(f))]
group_df['group'] = [reader.headers()[file]['Participants']['CHI']['group'] for file in range(len(f))]
group_df['participants'] = [reader.participants(by_files=True)[file] for file in range(len(f))]
group_df['mot_edu'] = [reader.headers()[file]['Participants']['CHI']['education'] for file in range(len(f))]
group_df['situation'] = [reader.headers()[file]['Situation'] for file in range(len(f))]
group_df['type'] = [reader.headers()[file]['Types'] for file in range(len(f))]

group_df

# reader.tokens(participants='CHI', by_files=True)[35]

In [ ]:
reader[3].headers()

In [ ]:
reader[0].headers()[0]['Situation'] if reader[0].headers()[0]['Situation'] else 0

if reader[0].headers()[0]['Situation']: print(reader[0].headers()[0]['Situation'])
